# 문서 요약

## 개요

- 사용 데이터: 신문 기사, 사설, 법률

- 설계
| 태스크 | 기술 |
|----------------|-----------------------------|
| 데이터 전처리 | 데이터 로드, 전처리 |
| 사용 모델 | KoBART, Qwen3-4b  |
| 훈련 | 정식 Fine-Tuning, QLora Fine-Tuning |

# 1. 데이터 전처리

In [ ]:
!pip install datasets
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 11.2 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [konlpy]


In [ ]:
!pip uninstall -y transformers tokenizers accelerate peft datasets safetensors

Found existing installation: transformers 4.48.3
Uninstalling transformers-4.48.3:
  Successfully uninstalled transformers-4.48.3
Found existing installation: tokenizers 0.21.0
Uninstalling tokenizers-0.21.0:
  Successfully uninstalled tokenizers-0.21.0
Found existing installation: accelerate 0.34.2
Uninstalling accelerate-0.34.2:
  Successfully uninstalled accelerate-0.34.2
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: datasets 2.21.0
Uninstalling datasets-2.21.0:
  Successfully uninstalled datasets-2.21.0
Found existing installation: safetensors 0.4.5
Uninstalling safetensors-0.4.5:
  Successfully uninstalled safetensors-0.4.5


In [ ]:
!pip install -U \
  "transformers==4.48.3" \
  "tokenizers==0.21.0" \
  "accelerate==0.34.2" \
  "peft==0.13.2" \
  "datasets==2.21.0" \
  "safetensors==0.4.5"

  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached accelerate-0.34.2-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.13.2-py3-none-any.whl.metadata (13 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached safetensors-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Using cached accelerate-0.34.2-py3-none-any.whl (324 kB)
Using cached peft-0.13.2-py3-none-any.whl (320 kB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
Using cached safetensors-0.4.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (434 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [peft]5/6 [peft]erate]s]


In [ ]:
!pip install -U bitsandbytes trl

  Using cached accelerate-1.12.0-py3-none-any.whl.metadata (19 kB)
  Using cached datasets-4.4.2-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.1 MB/s  0:00:05m0:00:0100:01
Using cached accelerate-1.12.0-py3-none-any.whl (380 kB)
Using cached datasets-4.4.2-py3-none-any.whl (512 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 11.3 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.2 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/6 [transformers]
      Successfully uninstalled transformers-4.48.3━━━━━━━━━━━━ 1/6 [transformers]
  Attempting uninstall: datasetsm━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip -q install soxr

In [ ]:
!pip -q install evaluate rouge_score

In [ ]:
# 파일 경로 및 파일 읽기 라이브러리
from pathlib import Path
from dataclasses import dataclass
import random
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from typing import Dict, List, Any, Tuple, Optional

# 파이토치 라이브러리
import torch

# 허깅페이스 관련 라이브러리
from datasets import (
    load_dataset,
    Dataset,
    DatasetDict
)
from transformers import (
    BartTokenizerFast,
    DataCollatorForSeq2Seq,
    BartForConditionalGeneration,
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig
)
from trl import SFTTrainer, SFTConfig
from peft import (
    LoraConfig,
    PeftModel
    )
import evaluate

# 훈련 시 시각화
from tqdm import tqdm

In [ ]:
"""
Config 클래스 정의

- 상수 변수 정의
- @dataclass 데코레이터 추가
"""
@dataclass
class Config:
    # 데이터 폴더 생성
    root = Path(".")
    raw_dir = root / "data"
    data_dir = raw_dir / "summarization"
    model_dir = root / "models"

    # kobart 데이터셋용 변수 선언
    text_max_len = 512
    summary_max_len = 256

    # 시스템 프롬프트
    sft_prompt = "다음 문서를 핵심만 유지하며 간결한 한국어로 요약하시오."

    # qwen 데이터셋용 길이 제한
    qwen_max_char = 2500
    qwen_max_sumary = 600

    # 랜덤 추출용 변수
    seed = 42

    # 학습용 변수 선언: 학습률, 에포크, 인내심, 최소 개선 폭
    lr = 5e-5
    epochs = 1

    # 디바이스 설정
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # 후처리 함수 설정
    def __post_init__(self):
        self.raw_dir.mkdir(parents=True, exist_ok=True)
        self.data_dir.mkdir(parents=True, exist_ok=True)
        self.model_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# 변수 생성 및 폴더 생성
cfg = Config()

In [ ]:
"""
데이터 다운로드

- 미션 페이지에서 데이터 다운로드
"""
with zipfile.ZipFile(f"{str(cfg.raw_dir)}/summarization.zip") as z:
    z.extractall(path=str(cfg.raw_dir))

In [ ]:
# JSON 파일 로드 함수
def load_json(json_path: Path) -> Dict:
    with open(str(json_path), "r", encoding="utf-8") as j:
        data = json.load(j)
    return data

In [ ]:
# JSON 파일 저장 함수
def save_jsonl(samples: List[Dict[str, str]], out_path: Path) -> None:
    out_path.parent.mkdir(parents=True, exist_ok=True)
    with out_path.open("w", encoding="utf-8") as f:
        for s in samples:
            f.write(json.dumps(s, ensure_ascii=False) + "\n")

In [ ]:
# json 데이터 불러오기
raw_train_edit = load_json(cfg.data_dir / "train_original_editorial.json")
raw_train_law = load_json(cfg.data_dir / "train_original_law.json")
raw_train_news = load_json(cfg.data_dir / "train_original_news.json")
raw_val_edit = load_json(cfg.data_dir / "valid_original_editorial.json")
raw_val_law = load_json(cfg.data_dir / "valid_original_law.json")
raw_val_news = load_json(cfg.data_dir / "valid_original_news.json")

In [ ]:
# 훈련 데이터 샘플 출력
print(f"원문: {' '.join([c.get("sentence") for d in raw_train_edit["documents"][0]["text"] for c in d if c.get("sentence")])}")
print(f"요약본: {' '.join(raw_train_edit["documents"][0]["abstractive"])}")
print(f"훈련 데이터 전체 길이: {len(raw_train_edit["documents"]) + len(raw_train_law["documents"]) + len(raw_train_news["documents"])}")
print(type(raw_train_edit))

원문: 이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 '공생발전'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다. 최근 미국 프랑스 벨기에 등에서 부유세가 거론되고 독일조차 2년간 한시적으로 5%의 자산세를 거둬 약 155조원을 마련하자는 논의가 있었다. 이런 흐름에 한국만 동떨어져 있기는 어려운 게 글로벌 시대의 특징이다. 항간에는 이번 회동 후 삼성을 비롯해 몇몇 그룹이 노블레스 오블리주 방안을 준비하고 있다는 말이 나도는데 대통령의 강요나 포퓰리즘에 의한 압박보다 자발적 문화로 만들어가야 효과가 큰 법이다. 그런 면에서 재계에 적절한 방안 마련을 맡기고 정치권이나 여론은 너무 압박하지 말고 시간을 줘야 한다. 국가채무 문제로 글로벌 경기 침체 우려가 큰 상황에서 기업들은 '생존'에 큰 부담을 느끼고 있기 때문이다. 이날 전경련에 따르면 30대 그룹은 올해 고용 12만4000명, 투자 114조원 등 '선물'을 준비했다. 세계적인 더블딥이 우려되는 상황에서 공격경영이 어렵겠지만 연초 한 번 발표한 내용을 약간 수정해 내놓은 전경련의 행태는 답답하다. 설립 50주년이 됐으면 좀 더 창의적이고 유연하게 바뀔 때도 됐다. 허창수 전경련 회장은 "대기업ㆍ중소기업이 서로 공생하고 발전할 수 있도록 노력하겠다. 기업이 사회적 책임을 다하겠다"는 원론적인 발언에 그쳐 전경련 특유의 무미건조함을 드러냈다. 한편 이건희 삼성전자 회장은 "중소기업계 협력을 강화해 국제적으로

In [ ]:
# 검증 데이터 샘플 출력
print(f"원문: {' '.join([c.get("sentence") for d in raw_val_edit["documents"][0]["text"] for c in d if c.get("sentence")])}")
print(f"요약본: {' '.join(raw_val_edit["documents"][0]["abstractive"])}")
print(f"훈련 데이터 전체 길이: {len(raw_val_edit["documents"]) + len(raw_val_law["documents"]) + len(raw_val_news["documents"])}")
print(type(raw_val_edit))

원문: 더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 '조국 사태'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 '조국 정국'을 거치며 분출된 '공정'과 '정의'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸움에 매몰된 구시대 인물들을 과감히 물갈이하라는 게

In [ ]:
# 데이터 통합 함수 생성
def unifing_data(*json_data: Tuple) -> List[Dict[str, str]]:
    """
    각 분할 파일을 하나로 정리

    Arg:
        Tuple: json 데이터 튜플

    Returns:
        unified: 각 원소는 {'text': str, 'summary': str}
    """
    pairs_list = []
    for json_f in json_data:
        if isinstance(json_f, dict):
            for text in json_f["documents"]:
                string_d = " ".join([d.get("sentence") for i in text["text"] for d in i if d.get("sentence")])
                summary_d = text["abstractive"]
                summary_d = " ".join(summary_d)
                pairs_list.append({"text": string_d, "summary": summary_d})

        else:
            raise TypeError(f"Unsupported JSON root type: {type(json_f)}")


    # 검증
    normalized_pairs: List[Dict[str, str]] = []
    for item in pairs_list:

        # 불량 데이터 거르기
        full_text = item.get("text", None)
        summary_text = item.get("summary", None)

        if full_text is None or summary_text is None:
            continue

        # 문자열이 아니면 강제로 변환
        normalized_pairs.append({"text": str(full_text), "summary": str(summary_text)})

    return normalized_pairs

In [ ]:
# 훈련 데이터 통합
train_normalized = unifing_data(raw_train_edit, raw_train_law, raw_train_news)

In [ ]:
# 정규화 데이터 예시 출력
print(f"통합 훈련 데이터 샘플: {train_normalized[0]}")

# 누락 샘플이 없는지 확인
print(f"통합 훈련 데이터 길이: {len(train_normalized)}")

통합 훈련 데이터 샘플: {'text': '이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 \'공생발전\'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다. 최근 미국 프랑스 벨기에 등에서 부유세가 거론되고 독일조차 2년간 한시적으로 5%의 자산세를 거둬 약 155조원을 마련하자는 논의가 있었다. 이런 흐름에 한국만 동떨어져 있기는 어려운 게 글로벌 시대의 특징이다. 항간에는 이번 회동 후 삼성을 비롯해 몇몇 그룹이 노블레스 오블리주 방안을 준비하고 있다는 말이 나도는데 대통령의 강요나 포퓰리즘에 의한 압박보다 자발적 문화로 만들어가야 효과가 큰 법이다. 그런 면에서 재계에 적절한 방안 마련을 맡기고 정치권이나 여론은 너무 압박하지 말고 시간을 줘야 한다. 국가채무 문제로 글로벌 경기 침체 우려가 큰 상황에서 기업들은 \'생존\'에 큰 부담을 느끼고 있기 때문이다. 이날 전경련에 따르면 30대 그룹은 올해 고용 12만4000명, 투자 114조원 등 \'선물\'을 준비했다. 세계적인 더블딥이 우려되는 상황에서 공격경영이 어렵겠지만 연초 한 번 발표한 내용을 약간 수정해 내놓은 전경련의 행태는 답답하다. 설립 50주년이 됐으면 좀 더 창의적이고 유연하게 바뀔 때도 됐다. 허창수 전경련 회장은 "대기업ㆍ중소기업이 서로 공생하고 발전할 수 있도록 노력하겠다. 기업이 사회적 책임을 다하겠다"는 원론적인 발언에 그쳐 전경련 특유의 무미건조함을 드러냈다. 한편 이건희 삼성전

In [ ]:
# 검증 데이터 통합
val_normalized = unifing_data(raw_val_edit, raw_val_law, raw_val_news)

In [ ]:
# 정규화 데이터 예시 출력
print(f"통합 검증 데이터 샘플: {val_normalized[0]}")

# 누락 샘플이 없는지 확인
print(f"통합 검증 데이터 길이: {len(val_normalized)}")

통합 검증 데이터 샘플: {'text': '더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 \'조국 사태\'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 \'조국 정국\'을 거치며 분출된 \'공정\'과 \'정의\'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대중의 이해와 요구를 잘 대표하는 정치인을 뽑는 행위다. 민생을 외면하며 낡은 이념과 진영 싸

In [ ]:
# 통합 데이터 저장
save_jsonl(train_normalized, cfg.data_dir / "train_normalized.jsonl")
save_jsonl(val_normalized, cfg.data_dir / "val_normalized.jsonl")
print("통합 데이터 파일 저장 완료!")

통합 데이터 파일 저장 완료!


## EXTRA: Qwen용 데이터셋

In [ ]:
# qwen용 데이터셋을 만들기 위한 데이터 샘플 생성 함수
def make_sft_sample(
        text: str,
        summary: str,
        instruction: str=cfg.sft_prompt,
        max_input_len: int=cfg.qwen_max_char,
        max_output_len: int=cfg.qwen_max_sumary
        ) -> Dict[str, str]:
    """
    Qwen SFT 학습 샘플 생성
    """
    # 길이 컷 (OOM 방지)
    if len(text) > max_input_len:
        text = text[:max_input_len]

    if len(summary) > max_output_len:
        summary = summary[:max_output_len]

    return {
        "instruction": instruction,
        "input": text,
        "output": summary,
    }


In [ ]:
# 데이터 변환용 리스트 초기화
qwen_train_processed = []
qwen_val_processed = []

for data in train_normalized:
    text = data["text"]
    summary = data["summary"]

    one_line = make_sft_sample(text, summary)
    qwen_train_processed.append(one_line)

print("훈련 데이터 변환 완료")

for data in val_normalized:
    text = data["text"]
    summary = data["summary"]

    one_line = make_sft_sample(text, summary)
    qwen_val_processed.append(one_line)

print("검증 데이터 변환 완료")

훈련 데이터 변환 완료
검증 데이터 변환 완료


In [ ]:
# qwen 데이터 예시 출력
print(f"통합 훈련 데이터 샘플: {qwen_train_processed[0]}")

# 누락 샘플이 없는지 확인
print(f"통합 훈련 데이터 길이: {len(qwen_train_processed)}")

통합 훈련 데이터 샘플: {'instruction': '다음 문서를 핵심만 유지하며 간결한 한국어로 요약하시오.', 'input': '이명박 대통령이 어제 30대 그룹 총수를 모아놓고 "시대적 요구는 역시 총수가 앞장서야 한다. 이미 상당한 변화의 조짐이 있다는 것을 고맙게 생각한다. 총수들께서 직접 관심을 가져주시면 빨리 전파돼 긍정적인 평가를 받을 수 있다고 본다"고 말했다. 언뜻 보아 무슨 말인지 불분명하나 이 대통령이 지난 8ㆍ15 연설 후 정몽준 의원, 정몽구 현대차 회장이 각각 2000억원과 5000억원을 기부한 사실과 \'공생발전\'이란 화두를 연결하면 금방 짐작이 간다. 다른 그룹 총수들도 좀 나서라고 은근히 떠민 것이다. 이 대통령은 기부에 대한 후속 선언이 나오지 않은 탓인지 총수들의 사회공헌 방안에 불만을 표시했다는 후문이다. 최근 미국 프랑스 벨기에 등에서 부유세가 거론되고 독일조차 2년간 한시적으로 5%의 자산세를 거둬 약 155조원을 마련하자는 논의가 있었다. 이런 흐름에 한국만 동떨어져 있기는 어려운 게 글로벌 시대의 특징이다. 항간에는 이번 회동 후 삼성을 비롯해 몇몇 그룹이 노블레스 오블리주 방안을 준비하고 있다는 말이 나도는데 대통령의 강요나 포퓰리즘에 의한 압박보다 자발적 문화로 만들어가야 효과가 큰 법이다. 그런 면에서 재계에 적절한 방안 마련을 맡기고 정치권이나 여론은 너무 압박하지 말고 시간을 줘야 한다. 국가채무 문제로 글로벌 경기 침체 우려가 큰 상황에서 기업들은 \'생존\'에 큰 부담을 느끼고 있기 때문이다. 이날 전경련에 따르면 30대 그룹은 올해 고용 12만4000명, 투자 114조원 등 \'선물\'을 준비했다. 세계적인 더블딥이 우려되는 상황에서 공격경영이 어렵겠지만 연초 한 번 발표한 내용을 약간 수정해 내놓은 전경련의 행태는 답답하다. 설립 50주년이 됐으면 좀 더 창의적이고 유연하게 바뀔 때도 됐다. 허창수 전경련 회장은 "대기업ㆍ중소기업이 서로 공생하고 발전할 수 있도록 노력하겠다. 기업이 사회적 책임을

In [ ]:
# qwen 데이터 예시 출력
print(f"통합 검증 데이터 샘플: {qwen_val_processed[0]}")

# 누락 샘플이 없는지 확인
print(f"통합 검증 데이터 길이: {len(qwen_val_processed)}")

통합 검증 데이터 샘플: {'instruction': '다음 문서를 핵심만 유지하며 간결한 한국어로 요약하시오.', 'input': '더불어민주당 이해찬 대표가 30 일 오후 국회에서 기자간담회를 열고 조국 전 법무부 장관 사태와 관련해 "국민 여러분께 매우 송구하다"고 밝혔다. 더불어민주당 이해찬 대표가 30 일 기자간담회를 열고 \'조국 사태\'와 관련, "국민 여러분께 매우 송구하다"는 입장을 밝혔다. 이 대표는 "검찰 개혁이란 대의에 집중하다 보니, 국민 특히 청년이 느꼈을 불공정에 대한 상대적 박탈감, 좌절감을 깊이 있게 헤아리지 못했다"며 "여당 대표로서 무거운 책임감을 느낀다"고 머리를 숙였다. 조국 전 법무부 장관이 14 일 사퇴한 이후 이 대표가 당 안팎의 쇄신 요구에 대해 입장을 표명한 것은 이번이 처음이다. 청와대와 여당은 \'조국 정국\'을 거치며 분출된 \'공정\'과 \'정의\'의 민심을 받들어 검찰 개혁에 매진하겠다면서도 두 달간 극심한 분열과 갈등을 초래한데 대해선 진지하게 성찰하는 모습을 보이지 않았다. 그나마 초선인 이철희 의원이 "당이 대통령 뒤에 비겁하게 숨어 있었다"고 비판했고, 표창원 의원은 "책임을 느끼는 분들이 각자 형태로 그 책임감을 행동으로 옮겨야 할 때"라고 지적했다. 뒤늦게나마 이 대표가 자성의 목소리를 내긴 했으나 당 안팎의 쇄신 요구에 어떻게 응할지 구체적 플랜을 제시하지 못해 여전히 안이하다는 지적도 나온다. 이 대표는 28 일 윤호중 사무총장을 단장으로 하는 총선기획단을 발족했고 조만간 인재영입위원회도 출범시킬 계획이라고 밝혔다. 이 대표는 "민주당의 가치를 공유하는 참신한 인물을 영입해 준비된 정책과 인물로 승부하겠다"고 다짐했다. 하지만 당 일각에선 "총선기획단장을 비롯한 당직 인선부터 쇄신 의지를 보여야 한다"는 비판의 목소리가 나온다. 무조건 물러나는 게 능사는 아니지만 국정 혼선을 초래한 데 대해 당 지도부가 겸허하게 책임지는 모습을 보이는 게 쇄신의 출발점이 돼야 한다는 지적도 있다. 선거는 대

In [ ]:
# Qwen용 데이터 저장
save_jsonl(qwen_train_processed, cfg.data_dir / "qwen_train_processed.jsonl")
save_jsonl(qwen_val_processed, cfg.data_dir / "qwen_val_processed.jsonl")
print("Qwen용 데이터 파일 저장 완료!")

Qwen용 데이터 파일 저장 완료!


# 2. 데이터셋

## 2.1. KoBART

In [ ]:
# 저장한 데이터셋 불러오기 (메모리 절약)
kobart_ds = load_dataset(
    "json",
    data_files={
        "train": str(cfg.data_dir / "train_normalized.jsonl"),
        "validation": str(cfg.data_dir / "val_normalized.jsonl"),
    }
)

Generating train split: 325072 examples [00:01, 215016.60 examples/s]
Generating validation split: 40134 examples [00:00, 206946.73 examples/s]


In [ ]:
# train_normalized와 val_normalized로 Dataset 객체 생성
# 훈련 시간 단축을 위한 데이터셋 랜덤 셔플 및 부분 선별
train_dataset = kobart_ds["train"].shuffle(seed=cfg.seed).select(range(50000))
val_dataset = kobart_ds["validation"].shuffle(seed=cfg.seed).select(range(2000))

# DatasetDict 형태로 통합 (훈련/검증)
kobart_dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [ ]:
# KoBART용 Fast Tokenizer 사용
kobart_id = "gogamza/kobart-base-v1"
kobart_tokenizer = BartTokenizerFast.from_pretrained(kobart_id)

# KoBART용 collate_fn 불러오기
kobart_collator = DataCollatorForSeq2Seq(tokenizer=kobart_tokenizer)

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'BartTokenizerFast'.


In [ ]:
# 데이터샘플 토큰화 함수
def kobart_tokenize_function(
        example: Dict[str, List[str]]
        ) -> Dict[str, List[List[int]]]:
    model_inputs = kobart_tokenizer(
        example["text"],
        max_length=cfg.text_max_len,
        truncation=True
    )
    labels = kobart_tokenizer(
        text_target=example["summary"],
        max_length=cfg.summary_max_len,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# 토큰화 함수 적용
kobart_dataset = kobart_dataset.map(kobart_tokenize_function, batched=True)

Map: 100%|██████████| 2000/2000 [00:00<00:00, 6093.40 examples/s]


## 2.2. Qwen3

In [ ]:
# 저장한 데이터셋 불러오기 (메모리 절약)
qwen_ds = load_dataset(
    "json",
    data_files={
        "train": str(cfg.data_dir / "qwen_train_processed.jsonl"),
        "validation": str(cfg.data_dir / "qwen_val_processed.jsonl"),
    }
)

Generating train split: 325072 examples [00:01, 220495.80 examples/s]
Generating validation split: 40134 examples [00:00, 187610.63 examples/s]


In [ ]:
# qwen_train_processed와 qwen_val_processed로 Dataset 객체 생성
# 훈련 시간 단축을 위한 데이터셋 랜덤 셔플 및 부분 선별
qwen_train_dataset = qwen_ds["train"].shuffle(seed=cfg.seed).select(range(20000))
qwen_val_dataset = qwen_ds["validation"].shuffle(seed=cfg.seed).select(range(1000))

# DatasetDict 형태로 통합 (훈련/검증)
qwen_dataset = DatasetDict({
    "train": qwen_train_dataset,
    "validation": qwen_val_dataset
})

In [ ]:
qwen_id = "Qwen/Qwen3-4B"
qwen_tokenizer = AutoTokenizer.from_pretrained(qwen_id, use_fast=True)

if qwen_tokenizer.pad_token is None:
    qwen_tokenizer.pad_token = qwen_tokenizer.eos_token

In [ ]:
def qwen_format_dataset_batch(batch):
    texts = []
    for inst, inp, out in zip(batch["instruction"], batch["input"], batch["output"]):
        texts.append(
            "### Instruction:\n"
            f"{inst}\n\n"
            "### Input:\n"
            f"{inp}\n\n"
            "### Output:\n"
            f"{out}"
        )
    return {"text": texts}

In [ ]:
qwen_dataset = qwen_dataset.map(qwen_format_dataset_batch, batched=True)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 37252.24 examples/s]


# 3. 모델 파인튜닝

## 3.1. KoBART 파인튜닝

In [ ]:
# KoBART 요약 모델 로드
kobart_model = BartForConditionalGeneration.from_pretrained(kobart_id).to(cfg.device)

# Hugging Face Trainer의 학습 설정(하이퍼파라미터) 묶음
kobart_args = TrainingArguments(
    # output_dir: 학습 산출물 저장 경로
    # 체크포인트(중간 저장), 최종 모델, 학습 로그 등이 이 경로에 저장
    output_dir=str(cfg.model_dir / "kobart_summarizer"),
    # 디바이스(GPU) 1개당 학습 배치 크기
    per_device_train_batch_size=2,
    # 디바이스 1개당 평가 배치 크기
    per_device_eval_batch_size=2,
    # 학습률
    learning_rate=cfg.lr,
    # 에포크 수
    num_train_epochs=1,
    # 가중치 감쇠(L2 정규화 성격)
    # 과적합을 억제하고, 파라미터가 지나치게 커지는 것을 방지
    weight_decay=0.01,
    # 평가 수행 기준
    # "steps": 일정 step마다 평가 수행
    # "epoch": epoch 끝날 때마다 평가 수행
    eval_strategy="steps",
    # 몇 step마다 평가할지 지정하는 파라미터
    eval_steps=500,
    # 몇 step마다 로그를 출력할지 지정하는 파라미터
    logging_steps=100,
    # save_steps: 몇 step마다 체크포인트를 저장하는 파라미터
    save_steps=500,
    # 체크포인트 최대 개수 제한
    save_total_limit=2,
    # fp16: mixed precision(반정밀도) 사용 여부
    # GPU가 있을 때  VRAM 사용량 감소
    fp16=torch.cuda.is_available(),
    # report_to: 외부 로깅 도구 사용 여부
    report_to="none",
)


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


In [ ]:
# Hugging Face Trainer 생성
kobart_trainer = Trainer(
    model=kobart_model,
    args=kobart_args,
    train_dataset=kobart_dataset["train"],
    eval_dataset=kobart_dataset["validation"],
    data_collator=kobart_collator,
    tokenizer=kobart_tokenizer,
)

# 학습 시작
kobart_trainer.train()


# 학습된 모델 저장
# KoBART는 모델(가중치) + 토크나이저(단어사전/토큰화 규칙)가 "세트"로 움직이므로 둘 다 저장
kobart_trainer.save_model(str(cfg.model_dir / "kobart_summarizer"))

# 토크나이저 vocab, merges, tokenizer_config 등 저장
kobart_tokenizer.save_pretrained(str(cfg.model_dir / "kobart_summarizer"))

print("KoBART 학습 및 저장 완료")


/tmp/ipykernel_1379/2538476431.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  kobart_trainer = Trainer(


Step,Training Loss,Validation Loss
500,2.086300,1.803897
1000,1.899900,1.750412
1500,1.884500,1.721366
2000,1.911400,1.714548
2500,1.722400,1.715675
3000,1.904600,1.676376
3500,1.776300,1.681219
4000,1.776000,1.666294
4500,1.724300,1.750922
5000,1.741000,1.647183


/home/ahnhs2k/pytorch-demo/.venv/lib/python3.12/site-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


KoBART 학습 및 저장 완료


## 3.2. Qwen QLora 파인튜닝

In [ ]:
"""
BitsAndBytesConfig: QLoRA(4bit) 로딩을 위한 양자화 설정

Base Model 가중치(weight)를 4bit로 양자화해서 GPU 메모리를 크게 줄이고
LoRA 어댑터만 학습(업데이트)하는 방식으로 fine-tuning을 수행한다.

BitsAndBytesConfig는 bitsandbytes 라이브러리를 이용해
모델을 4bit/8bit 형태로 로드할 때 필요한 옵션들을 담는 설정 객체다.
"""

bnb_config = BitsAndBytesConfig(
    # load_in_4bit=True
    # 모델의 base weight를 4bit 정밀도로 로드한다.
    load_in_4bit=True,
    # 실제 연산(Compute)은 어떤 dtype으로 할지 지정.
    bnb_4bit_compute_dtype=torch.bfloat16,
    # bnb_4bit_quant_type="nf4"
    # 4bit 양자화 방식(Quantization Type)을 지정한다.
    # "nf4"(NormalFloat4)는 QLoRA 논문/실무에서 가장 많이 쓰는 방식 중 하나로, 단순 4bit보다 정보 보존이 좋아서 품질 손실을 줄이기 유리하다.
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_use_double_quant=True
    # Double Quantization(이중 양자화)을 사용할지 지정
    # 4bit로 줄인 가중치를 다시 한 번 더 효율적으로 압축/표현해서 VRAM을 추가로 절약하는 옵션
    bnb_4bit_use_double_quant=True,
)


In [ ]:
# Qwen 모델 로드 (4bit QLoRA 학습 준비)
qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_id,
    quantization_config=bnb_config,
    device_map="auto",
)

# 메모리 절약 설정 Gradient Checkpointing 활성화
# gradient checkpointing은 학습 시 중간 activation(중간 계산 결과)을 전부 저장하지 않고 일부만 저장하는 방식
qwen_model.gradient_checkpointing_enable()

# 메모리 절약 설정 use_cache 비활성화
# use_cache는 "추론(inference)"에서 속도를 올리기 위해 past_key_values(Attention 캐시)를 저장하는 기능
# 하지만 "학습(training)"에서는 이 캐시가 오히려 GPU 메모리를 크게 잡아먹고, gradient checkpointing과 충돌/경고를 유발하는 경우가 많으므로 False
qwen_model.config.use_cache = False


Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.82s/it]


In [ ]:
"""
LoRAConfig: QLoRA(LoRA + 4bit) 학습을 위한 어댑터 설정
LoRA(Parameter-Efficient Fine-Tuning)는
- 원본(Base) 모델의 가중치는 동결(freeze)하고
- 일부 선형 레이어에만 작은 저차원 행렬(LoRA 어댑터)을 추가하여
  학습 비용(VRAM/시간)을 크게 줄이면서도 성능을 확보하는 기법

QLoRA에서는
- Base Model: 4bit 양자화된 상태로 유지
- LoRA Adapter: FP16/FP32로 학습
이 조합을 사용
"""

lora_config = LoraConfig(
    # r (rank): LoRA의 저차원(rank) 크기
    # - LoRA 어댑터는 (원래 가중치 행렬 W 대신)
    #   W + (A @ B) 형태로 업데이트를 수행
    # - 이때 A, B의 차원을 결정하는 것이 r

    # - r이 클수록 표현력 증가 (더 많은 정보를 학습 가능)

    # - r이 작을수록 학습은 가벼워지지만 표현력이 제한됨
    r=8,
    # lora_alpha: LoRA scaling 계수
    # - LoRA 업데이트 (A @ B)에 곱해지는 스케일링 값
    # - 실제 업데이트는 대략:
    #   (lora_alpha / r) * (A @ B)

    # - 관례적으로 lora_alpha는 r의 2배 정도로 설정
    #   (r=8 -> alpha=16, r=16 -> alpha=32 등)

    # - 너무 작으면 학습 효과가 약해지고, 너무 크면 학습이 불안정해질 수 있다.
    lora_alpha=16,
    # lora_dropout: LoRA 어댑터에 적용되는 dropout 비율
    lora_dropout=0.05,
    # bias: bias 파라미터 처리 방식
    # - "none": bias는 학습하지 않음 (가장 일반적인 설정)
    # - "all": 모든 bias를 학습
    # - "lora_only": LoRA가 적용된 레이어의 bias만 학습

    # - QLoRA에서는 보통 "none"을 사용해 학습 파라미터 수를 최소화
    bias="none",
    # task_type: LoRA를 적용할 모델의 태스크 유형
    # - "CAUSAL_LM": Causal Language Modeling
    #   (다음 토큰 예측 기반 모델, Qwen/LLaMA 계열)

    # - 이 값에 따라 LoRA가 내부적으로
    #   어떤 forward 구조를 기준으로 동작할지 결정된다.
    task_type="CAUSAL_LM",
    # target_modules: LoRA를 적용할 레이어 이름들
    # - LoRA는 "모든 레이어"가 아니라,
    #   특정 선형(linear) 레이어에만 삽입된다.

    # - 아래 모듈들은 Qwen 계열 Transformer에서 주로 사용되는 핵심 투영 레이어들:

    #   q_proj, k_proj, v_proj:
    #     - Attention의 Query / Key / Value projection

    #   o_proj:
    #     - Attention 출력 projection

    #   gate_proj, up_proj, down_proj:
    #     - FFN(Feed-Forward Network, MLP) 내부의 projection 레이어들
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)


In [ ]:
qwen_args = SFTConfig(
    output_dir=str(cfg.model_dir / "qwen_summarizer_qlora"),
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8, # 여러 그레디언트 모아서 한 번에 업데이트
    learning_rate=2e-4,
    num_train_epochs=1,
    logging_steps=50,
    save_total_limit=2,
    save_steps=300,
    eval_strategy="steps",
    eval_steps=300,
    bf16=torch.cuda.is_available(),
    report_to="none",
    dataset_text_field="text",
)

In [ ]:
qwen_trainer = SFTTrainer(
    model=qwen_model,
    args=qwen_args,
    train_dataset=qwen_dataset["train"],
    eval_dataset=qwen_dataset["validation"],
    peft_config=lora_config,
)

qwen_trainer.train()

qwen_trainer.model.save_pretrained(str(cfg.model_dir / "qwen_summarizer_qlora"))
qwen_tokenizer.save_pretrained(str(cfg.model_dir / "qwen_summarizer_qlora"))

print("Qwen QLoRA 학습 및 저장 완료")

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
300,1.705500,1.685624,1.690506,1963708.000000,0.614289
600,1.703200,1.662069,1.666654,3933986.000000,0.617698
900,1.691400,1.647017,1.626818,5898205.000000,0.620226
1200,1.681700,1.636094,1.639056,7887692.000000,0.622147
1500,1.650900,1.627955,1.627508,9860540.000000,0.623774
1800,1.650000,1.622054,1.629847,11832016.000000,0.624774
2100,1.642100,1.618354,1.620977,13814710.000000,0.625449
2400,1.651400,1.617045,1.620867,15788845.000000,0.625590


Qwen QLoRA 학습 및 저장 완료


# 4. 테스트 및 평가

## 4.1. KoBART

In [ ]:
# 토크나이저 로드
kobart_tokenizer = AutoTokenizer.from_pretrained(str(cfg.model_dir / "kobart_summarizer"))

# 모델 로드 (Seq2Seq)
kobart_model = BartForConditionalGeneration.from_pretrained(str(cfg.model_dir / "kobart_summarizer")).to(cfg.device)
kobart_model.eval()

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_n

In [ ]:
# 추론 함수
def summarize_kobart(text: str, max_input_len: int=cfg.text_max_len, max_new_tokens: int=cfg.summary_max_len) -> str:
    inputs = kobart_tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_len,
    ).to(cfg.device)

    with torch.no_grad():
        output_ids = kobart_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True,
        )

    return kobart_tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
sample_text = kobart_ds["validation"][2000]["text"]
print("KoBART 요약:", summarize_kobart(sample_text))

KoBART 요약: 대한상공회의소 지속성장이니셔티브(SGI)는 8 일 하나의 산업을 둘러싸고 나뭇가지처럼 얽혀있는 연관 규제를 도식화한 '규제트리'를 공개하고 국내 신산업이 경쟁력을 갖추기 위해서는 대못규제, 중복규제, 소극규제 등 3 대 규제를 풀어야 한다고 주장했다.                                                                                                                                                                                                


## 4.2. Qwen

In [ ]:
# base tokenizer 로드
qwen_tokenizer = AutoTokenizer.from_pretrained(str(cfg.model_dir / "qwen_summarizer_qlora"), use_fast=True)
if qwen_tokenizer.pad_token is None:
    qwen_tokenizer.pad_token = qwen_tokenizer.eos_token
qwen_tokenizer.padding_side = "right"

# base model 로드 (4bit)
qwen_model = AutoModelForCausalLM.from_pretrained(
    qwen_id,
    quantization_config=bnb_config,
    device_map="auto",
)
qwen_model.eval()

# LoRA adapter 적용
qwen_model = PeftModel.from_pretrained(qwen_model, str(cfg.model_dir / "qwen_summarizer_qlora"))
qwen_model.eval()

Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.97s/it]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen3ForCausalLM(
      (model): Qwen3Model(
        (embed_tokens): Embedding(151936, 2560)
        (layers): ModuleList(
          (0-35): 36 x Qwen3DecoderLayer(
            (self_attn): Qwen3Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

In [ ]:
# qwen용 추론 함수
def summarize_qwen(text: str, instruction: str=cfg.sft_prompt, max_new_tokens: int=200) -> str:
    prompt = (
        "### Instruction:\n"
        f"{instruction}\n\n"
        "### Input:\n"
        f"{text}\n\n"
        "### Output:\n"
    )

    inputs = qwen_tokenizer(prompt, return_tensors="pt").to(qwen_model.device)

    with torch.no_grad():
        output_ids = qwen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )

    decoded = qwen_tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 모델 출력에서 프롬프트 부분 제거 (Output 이후만 보기)
    if "### Output:" in decoded:
        decoded = decoded.split("### Output:", 1)[-1].strip()

    return decoded.strip()

In [ ]:
print("Qwen 요약:", summarize_qwen(sample_text))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Qwen 요약: 대한상공회의소 지속성장이니셔티브(SGI)는 8일 하나의 산업을 둘러싸고 나뭇가지처럼 얽혀있는 연관 규제를 도식화한 '규제트리'를 공개하고 "이미 뒤처진 신산업 분야에서 경쟁국을 따라잡으려면 무엇보다 각종 규제를 정비하는 노력이 필요하다"고 주장했으며, 이중삼중으로 쳐져 있는 규제를 중점관리해야 한다고 제안했다.


## 4.3. ROUGE 평가

In [ ]:
# ROUGE metric 로더
rouge = evaluate.load("rouge")

def compute_rouge(
    dataset,                                 # validation 데이터셋: [{"text":..., "summary":...}, ...]
    summarize_fn,                            # summarize_kobart or summarize_qwen
    n_samples=200,                           # 너무 오래걸리니까 일부만 샘플링
    max_new_tokens=cfg.summary_max_len,      # 생성 길이 제한(모델별로 조절)
    seed=cfg.seed
):
    """
    validation set에서 ROUGE 점수를 계산

    Args:
        dataset: List[Dict] 형태 (text/summary 포함)
        summarize_fn: 요약 생성 함수 (입력 text -> 요약문 str)
        n_samples: 평가에 사용할 샘플 개수 (전체가 많으면 일부만 사용)
        max_new_tokens: generate()에서 생성 최대 길이 (모델 길이 폭주 방지)
        seed: 셔플 시드

    Returns:
        dict: rouge1/rouge2/rougeL/rougeLsum 점수
    """
    # 샘플링 (평가 시간 단축)
    rng = random.Random(seed)
    idxs = list(range(len(dataset)))
    rng.shuffle(idxs)
    idxs = idxs[:n_samples]

    preds = []
    refs = []

    # 예측 요약 생성
    for i in tqdm(idxs, desc="Generating summaries"):
        src = dataset[i]["text"]
        ref = dataset[i]["summary"]

        pred = summarize_fn(src, max_new_tokens=max_new_tokens)

        preds.append(pred)
        refs.append(ref)

    # ROUGE 계산
    scores = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
    return scores

In [ ]:
# kobart rouge 점수 계산
kobart_rouge_scores = compute_rouge(
    dataset=kobart_ds["validation"],
    summarize_fn=summarize_kobart,
)

print("[KoBART ROUGE]")
for k, v in kobart_rouge_scores.items():
    print(f"{k}: {v:.4f}")

Generating summaries: 100%|██████████| 200/200 [03:55<00:00,  1.18s/it]

[KoBART ROUGE]
rouge1: 0.4055
rouge2: 0.1575
rougeL: 0.3908
rougeLsum: 0.3895


In [ ]:
# kobart rouge 점수 계산
qwen_rouge_scores = compute_rouge(
    dataset=kobart_ds["validation"],
    summarize_fn=summarize_qwen,
    n_samples=100,
)

print("[Qwen ROUGE]")
for k, v in qwen_rouge_scores.items():
    print(f"{k}: {v:.4f}")

Generating summaries: 100%|██████████| 100/100 [16:42<00:00, 10.02s/it]

[Qwen ROUGE]
rouge1: 0.3210
rouge2: 0.1155
rougeL: 0.3033
rougeLsum: 0.3024


### 코멘트

- KoBART와 Qwen은 서로 다른 조건에서 훈련이 진행되었으며, 따라서 수치 비교는 정확하지 않습니다.
- 다만 KoBART는 훈련 시간 대비(KoBART는 약 25분, Qwen3-4B는 약 3시간 20분) 좋은 성능을 보였습니다.
- Qwen의 경우, 4B는 태스크 대비 과도하게 무거운 모델로 생각되며, 더 가벼운 모델이 바람직해 보입니다.